In [4]:
#!/usr/bin/env python
# coding: utf-8

# In[2]:


import numpy as np
#from termcolor import colored
from scipy.integrate import solve_ivp
from scipy.linalg import norm
from numpy import pi as pi
import scipy
from scipy import sparse
import scipy.sparse as sp
from scipy.sparse import csr_matrix
from numpy.random import choice, normal
from numpy import concatenate
from scipy.sparse import coo_matrix
from scipy.sparse import kron
import pandas as pd
from mpl_toolkits import mplot3d
from numba import jit
from scipy.linalg import expm
import numbers
from numpy import imag, conj, cos, sin
from numpy import sqrt, zeros, diag, random, real, array, complex_, transpose, conjugate, exp, matmul, real, dot, add, sum
import pandas as pd
from numpy import dot
from scipy.sparse import bsr_matrix

df=pd.read_csv('Positions_2000.txt') 




Gamma_r=1    #  2*pi*6e6  #reference
Omega=4*Gamma_r   #Gamma_r  #Driving frequency , What are the units?
N_atom=2000      #Number of atoms
Atom_cutoff=6







lambda_r  =1    # in units
Lambda_las=1     # in units    Gamma_r*2*pi/(wL/c_light)   #Laser wavelenght  , What are the units?
e_L=array([0,0,1]/sqrt(1))

Theta1=pi/2
Theta2=pi/2   #they got [0,1,0]

e_0=array([0,0,1]/sqrt(1))   #array([sin(Theta1)*cos(Theta2),sin(Theta1)*sin(Theta2),cos(Theta1)])

K_laser=(2*pi/Lambda_las)*array([1,0,0])
k0=2*pi/Lambda_las
Gamma_dec=Gamma_r

# Clebsch-Gordan coefficient

# Clebsch-Gordan coefficient
C00=1    

# Positions


## Oblate Variances matrix: Set the shape of the atoms cloud

Sigma_x=5 *lambda_r    
Sigma_y=0.05*lambda_r 
Sigma_z=Sigma_y     


Oblate_variances=zeros((3,3))

Oblate_variances[0][0]=Sigma_x**2
Oblate_variances[1][1]=Sigma_y**2
Oblate_variances[2][2]=Sigma_z**2

Miu_pos=[0 , 0, 0]

Pos_atoms=(random.multivariate_normal(Miu_pos, Oblate_variances, N_atom))

Pos_atoms=array(df)



Pos_atoms=Pos_atoms[Pos_atoms[:,0].argsort()]


#for j in range(N_atom):
#    Pos_atoms[j]=np.array([0.1*Lambda_las*j,0,0])
  
# 2*np.pi

#Pos_atoms
#Pos_atoms    



# Construct $I_{Lk}$ and $R_{Lk}$


# Construct I_Lk and R_Lk

eL_e0c=dot(e_L,conj(e_0))

conj_0=conj(e_0)

DotKL_rk=[]   #summarize k_L.r_k for all k

for j in range(N_atom):
    DotKL_rk.append(dot(Pos_atoms[j],K_laser))

DotKL_rk_exp=exp(1j*array(DotKL_rk))


ZL_k= DotKL_rk_exp*eL_e0c


ImL_k=imag(ZL_k)
RmL_k=real(ZL_k)


# Construct $G_{ij}$

# Construct the matrix R00
#t0=time.time()

@jit
def fun_Pkl0(k,l,k0=k0):
    
    VecR_kl=Pos_atoms[k]-Pos_atoms[l]
    R_kl=sqrt(dot(VecR_kl,VecR_kl))
    #R_kl=norm(VecR_kl)
    
    
    #Pkl0=dot(conj(e_0),VecR_kl)
    Pkl0=dot(conj_0,VecR_kl)/R_kl
    Pkl_0nC=(abs(Pkl0))**2
    
    M0kl=k0*R_kl
    
    return (1-Pkl_0nC)/M0kl+(1-3*Pkl_0nC)*(1j/(M0kl)**2-1/(M0kl)**3)
    


MatL=zeros((N_atom,N_atom),dtype='complex64')
MatR=zeros((N_atom,N_atom),dtype='complex64')


for i in range(N_atom):
    for j in range(i+1,N_atom):

        VecR_ij=Pos_atoms[i] -Pos_atoms[j]

        MatL[i][j]=exp(1j*k0*sqrt(dot(VecR_ij,VecR_ij)))
        MatR[i][j]=fun_Pkl0(i,j,k0)

                
MatR=MatR+MatR.T
MatL=MatL+(MatL.T)


Gij=(3*Gamma_dec/4)*MatL*MatR


#t1=time.time()

Gij=Gij + 1j*0.5*Gamma_r*np.identity(N_atom) 



Abs_Gij=abs(Gij)


ReG=real(Gij)
ImG=imag(Gij)

Collect    =0

#t0=time.time()




######### Set artificially the Rij=0

Gij=1j*imag(Gij)

############




# In[ ]:





# In[5]:


Sigma_eg=csr_matrix(np.array([[0,1],
                              [0,0]]),dtype="complex64")
Id_2LS  =csr_matrix(np.array([[1,0],
                              [0,1]]),dtype="complex64")

Zero_2LS=csr_matrix(np.array([[0,0],
                              [0,0]]),dtype="complex64")

state   =csr_matrix(np.array([[0,0],
                              [0,1]]),dtype="complex64")


Sigma_ee=csr_matrix(np.array([[1,0],
                              [0,0]]),dtype="complex64")

Sigma_egT=Sigma_eg.T    




def index_Gij(arr,Atom_cutoff):
    
    Ordered_N = list(np.argsort(arr)[::-1][:Atom_cutoff])
    
    return Ordered_N

Index_N=[]

Rem_iden=abs(Gij - 1j*0.5*Gamma_r*np.identity(N_atom))


for j in range(Rem_iden.shape[0]):
    
    test_list = list(Rem_iden[j])
    
    Index_N.append( [j]+list(index_Gij(test_list,Atom_cutoff-1))  )



    
    
    
#@jit
def tensor_id(ArrayL ):
    
    Tensor_id=kron(ArrayL[0],ArrayL[1])
    
    Array_id=Atom_cutoff*[Id_2LS]    
    
    for j in range(2,len(ArrayL)):
        
        Tensor_id=kron(Tensor_id,Array_id[j])
        
    
    Ten_s=csr_matrix(Tensor_id)
    
    csr_matrix.eliminate_zeros(Ten_s)  #remove zeros!!!
    
    return coo_matrix(Ten_s)



#@jit
def tensor_kj( k , j):   #tensor among k,j are Positions in the cluster after the G classification
                         #k is +sigma and j is -sigma 
    Array_id=Atom_cutoff*[Id_2LS]  
    
    Array_id[k]=Sigma_eg
    Array_id[j]=Sigma_egT
    
    Tensor_kj=kron(Array_id[0],Array_id[1])
    
    for j in range(2,len(Array_id)):
        
        Tensor_kj=kron(Tensor_kj,Array_id[j])
        
        csr_matrix.eliminate_zeros(csr_matrix(Tensor_kj))
        
        
            
    Ten_kj=csr_matrix( Tensor_kj ) 
        
    csr_matrix.eliminate_zeros(Ten_kj)
        
        
    
    return coo_matrix(Ten_kj)



#@jit
def tensor_k_ee( k  ):  #Excited state k is a Position in the cluster after the G classification
    
    Array_id=Atom_cutoff*[Id_2LS]  
    
    Array_id[k]=Sigma_ee
    
    Tensor_k    =kron(Array_id[0],Array_id[1])
    
    for j in range(2,len(Array_id)):
        
        Tensor_k=kron(Tensor_k,Array_id[j])
        
    
    
    Tensor_k=csr_matrix( Tensor_k ) 
    
    csr_matrix.eliminate_zeros(Tensor_k)
        
    
    return coo_matrix(Tensor_k)



#@jit
def tensor_k( k  ):  #sigmaplus k
    
    Array_id=Atom_cutoff*[Id_2LS]  
    
    Array_id[k]=Sigma_eg
    
    Tensor_k    =kron(Array_id[0],Array_id[1])
    
    for j in range(2,len(Array_id)):
        
        Tensor_k=kron(Tensor_k,Array_id[j])
        
    
    Tensor_k=csr_matrix( Tensor_k ) 
    
    csr_matrix.eliminate_zeros(Tensor_k)
        
    
    return coo_matrix(Tensor_k)



#@jit
def init_rho(Atom_cutoff=Atom_cutoff,state=state):
    
    state_fact=Atom_cutoff*[state]
    
    Rho_in=kron(state_fact[0],state_fact[1])
    
    for j in range(2,Atom_cutoff):
        
        Rho_in=kron(Rho_in,state_fact[j])
        
        
    Rho_in=csr_matrix( Rho_in ) 
    
    csr_matrix.eliminate_zeros(Rho_in)
    
    
        
    return coo_matrix(Rho_in)





#@jit
def Hamil_atom(index_center):
    
    Index_list=Index_N[index_center]
    Zero_gen=tensor_id(Atom_cutoff*[ Zero_2LS ])
    Z2=Zero_gen
    
    for k in range(Atom_cutoff):
        
        IL_k = Index_list[k]
        
        #print(IL_k)
        
        zL_k = ZL_k[IL_k]
        Ten_k= tensor_k(k)
        
        Z2   = Z2 -(Omega)*(zL_k*Ten_k  +conj(zL_k)*(Ten_k.T) )
        
    return Z2

Sigma_EGalt=(tensor_k(0)).reshape(1,4**Atom_cutoff)
Sigma_EEalt=(tensor_k_ee(0)).reshape(1,4**Atom_cutoff)
Sigma_Min_alt=(tensor_k(0).T).reshape(1,4**Atom_cutoff)


# In[ ]:





# In[25]:


Ivec=[4**Atom_cutoff-1]

Jvec=[0]
Vvec=[1]


Init_rho=(sparse.coo_matrix((Vvec,(Ivec,Jvec)),shape=(4**Atom_cutoff,1))) 


    #@jit
def f(t, yR,M_vect,b):
 
    MatA=M_vect@yR
    return MatA



def Hamil_effec(i_index, Expect_minus):   #this is just for the site labelled by index_center
    
    Vect_Gi=Gij[i_index]
    
    All_Ind_effective_drive=dot(Vect_Gi,Expect_minus.todense())[0,0]
    
    return All_Ind_effective_drive

#Hamil_effec(0, Sigma_m_rev[:,0])




def GdotMinus(Expect_minus):
    
    return (Expect_minus.T).dot(Gij)




Rho_t= sparse.lil_matrix((4**Atom_cutoff, N_atom),dtype="complex64")   #saves the density matrices for a given time t
for j in range(N_atom):
    Rho_t[:,j]=Init_rho


        
tfinal=10
Lenght=600
tspan=np.linspace(0,tfinal,Lenght)    #Here the time definition is important because it will tell you how to divide the elements



Sigma_m_rev=  sparse.lil_matrix((N_atom, len(tspan)),dtype="complex64")
Sigma_ee_rev= sparse.lil_matrix((N_atom, len(tspan)),dtype="complex64")



for j in range(N_atom):
    
    Sigma_m_rev[j,0] =Sigma_Min_alt.dot( Rho_t[:,j])[0,0]
    Sigma_ee_rev[j,0]=Sigma_EEalt.dot( Rho_t[:,j])[0,0]
    

#Sigma_m_rev[:,0][1,0]




def hamil_clust(index_center,ind_ref,Expect_minus):
    
    
    Index_list=Index_N[index_center]
    
    ind_ref=Index_list[ind_ref]
    
    Driv_clust=0
    
    for ind_list in Index_list:
        
        Driv_clust=Driv_clust+Gij[ind_ref, ind_list ]*Expect_minus[ind_list , 0 ]
    
    return Driv_clust
        

#Gmin[Index_list[j],0] 



def Effec_Hamil( index_center ,tind_):
    
    Expect_minus=Sigma_m_rev[:,tind_]
    
    Index_list=Index_N[index_center]
    
    Gmin=GdotMinus(Sigma_m_rev[:,tind_])
    
    H_effect = 0*tensor_k(0)
    
    for k in range(Atom_cutoff):
        H_effect = H_effect -(Gmin[0,Index_list[k]]  -  hamil_clust(index_center , k  ,Expect_minus)  ) * tensor_k(k)
    
    return (H_effect+conj(H_effect.T))




# In[14]:


def Part_H_L_R_LR(index_center):
    
    Array_id=Atom_cutoff*[Id_2LS] 
    Index_list=Index_N[index_center]
    Zero_gen=tensor_id(Atom_cutoff*[ Zero_2LS ])
    
    Z1=Zero_gen
    Z2=Zero_gen
    Z3=kron(Zero_gen,Zero_gen)
    
    
    Col_Decay=kron(Zero_gen,Zero_gen)
    
    Identity=tensor_id( Atom_cutoff*[Id_2LS])
    
    Col=Collect
    
    D_R = Zero_gen
    D_L = Zero_gen
    D_RL= kron(Zero_gen,Zero_gen)

    Hamil=Zero_gen
    
    Splus=Zero_gen
    
    
    Hamil_at=Hamil_atom(index_center)   
    Hamil_vect= -1j *(kron(Identity,  Hamil_at )-kron(Hamil_at.T,Identity))
    
    for k in range(Atom_cutoff):
        
        IL_k = Index_list[k]

        zL_k = ZL_k[IL_k]

        Ten_k= tensor_k(k)
        
        Ten_k_ee=tensor_k_ee(k)
        
        
        Splus=Splus+Ten_k
        
        Imkk=ImG[k,k]  
        
        
            #Z2   = Z2+ 1j*(Omega)*(zL_k*Ten_k  +conj(zL_k)*(Ten_k.T) )
        
        
        D_R   = D_R  - ( Imkk       )*Ten_k_ee
        D_L   = D_L  - ( Imkk       )*Ten_k_ee
        D_RL  = D_RL + ( 2 * Imkk    )*kron( Ten_k.T , Ten_k.T  )

        for j in range(k+1,Atom_cutoff):
            
            IL_j  =Index_list[j]

            Ten_kj= tensor_kj( k , j )
            
            ReGkj=ReG[IL_k,IL_j] 
            ImGkj=ImG[IL_k,IL_j]
            ReGjk=ReGkj   
            ImGjk=ImGkj
            
            
            Z1=Z1+ (1j*ReGkj-ImGkj)*(Ten_kj+ Ten_kj.T  )
            
            Ten_j=tensor_k(j)
            
            
            Tenk_Tenj=kron(Ten_k.T,Ten_j.T)

            Tenj_Tenk=kron(Ten_j.T,Ten_k.T)
            
            
            Z3=Z3+2*( ImGkj*Tenk_Tenj   +  ImGjk* Tenj_Tenk )

            #print( tensor_kj( k , j ).todense(), ReG[IL_k,IL_j],(IL_k,IL_j)  )
            
            #print(k,j)



            Hamil=Hamil+(1j*ReGkj)*(Ten_kj+ Ten_kj.T)

    Hamil=Hamil+Hamil_at                 
        
    L_R = Z2+Z1
    L_L =-(Z2.T)+conj(Z1.T) 
    L_RL=Z3
    

    #print(L_R.shape, L_L.shape, L_RL.shape)
    
    Vectorization=  ( kron(Identity,  L_R+D_R )+kron(L_L+D_L,Identity )+L_RL+D_RL ) #   + Col * Col_Decay  )

    
    return  Vectorization+Hamil_vect  #, Hamil #cl_sparse(Vectorization) #clean_sparse(Vectorization)


# In[ ]:


# In[ ]:





# In[ ]:





# In[ ]:





# In[ ]:





# In[3]:


# In[15]:


def Part_time_dep(index_center,tind_):

    Identity=tensor_id( Atom_cutoff*[Id_2LS])
    
    Hamil_at  =  Effec_Hamil( index_center ,tind_)  #Put zero to ignore mean-field 
    Hamil_vect= -1j *(kron(Identity,  Hamil_at )-kron(Hamil_at.T,Identity))
        
    return  Hamil_vect  #, Hamil #cl_sparse(Vectorization) #clean_sparse(Vectorization)


# In[ ]:





# In[16]:




Time_ind_hamil=[]   #Calculates all the time-independent Lindblandian vectorized

for cent_ind in range(N_atom):
    
    Time_ind_hamil.append(Part_H_L_R_LR(cent_ind))
    
    #print(cent_ind*100/N_atom)
    


# In[ ]:





# In[17]:




def rho_part(Rho_mt,inst_time):
    
    Result_fin=Rho_mt

    for cent_ind in range(N_atom):
        
        #Mat_dyn=Time_ind_hamil[cent_ind]+Part_time_dep(cent_ind,inst_time)
        
        Mat_dyn=Time_ind_hamil[cent_ind]+Part_time_dep(cent_ind,inst_time)
        
        sol_cent_ind = solve_ivp(f,[tspan[inst_time],tspan[inst_time+1]],list(array((Rho_mt[:,cent_ind].todense()).T,dtype="complex")[0])
                                 ,args=(Mat_dyn,None),dense_output=False , t_eval= [tspan[inst_time+1]])
        
        Result_fin[:,cent_ind]=sol_cent_ind.y
        
        #print(cent_ind)
        
    return Result_fin
        


# In[ ]:





# In[18]:


Rho0= sparse.lil_matrix((4**Atom_cutoff, N_atom),dtype="complex64")   #saves the density matrices for a given time t
for j in range(N_atom):
    Rho0[:,j]=Init_rho


# In[ ]:





# In[ ]:





# In[19]:


for tind_ in range(len(tspan)-1):
    
    Rho0=rho_part(Rho0,tind_)
    
    
    for j in range(N_atom):
    
        Sigma_m_rev[j ,tind_+1] = Sigma_Min_alt.dot( Rho0[:,j] )[0,0]
        Sigma_ee_rev[j,tind_+1] = Sigma_EEalt.dot(   Rho0[:,j] )[0,0]


        if tind_%120==0 and tind_>0:
        

            Tim0  = tspan.reshape(len(tspan),1)
            See0  = array(sum(Sigma_ee_rev,axis=0)).reshape(len(tspan),1)
            Smin0=  array(sum(Sigma_m_rev,axis=0)).reshape(len(tspan),1)
            Res_conc=np.concatenate((Tim0,See0,Smin0),axis=1)
            Res_conc=np.concatenate((Res_conc[0].reshape(1,len(Res_conc[0])),Res_conc),axis=0)
            file = open(str(N_atom)+"TestNew_Results"+"_Cluster_size_"+str(Atom_cutoff)+"_Omega_"+str(Omega)+"_Lax_"+str(Sigma_x)+"_Tspan_"+str(tspan[0])+"-"+str(tspan[tind_])+".txt", "w+")
            np.savetxt(file, Res_conc, delimiter=',')
            file.write('\n')
            file.write('\n')
            file.close()
        
        
    print(str((tind_+1)*100/len(tspan)))
    


# In[ ]:





# In[20]:


Tot_minus=[]
Tot_excit=[]

for j in range(len(tspan)):
    
    Tot_minus.append( sum(Sigma_m_rev[:,j]) )
    Tot_excit.append( sum(Sigma_ee_rev[:,j]) )
    


# In[ ]:





# In[ ]:





# In[ ]:





# In[ ]:





# In[22]:


#Texc=real(Tot_excit)
#Tx=real(Tot_minus)
#Ty=imag(Tot_minus)


# In[ ]:





# In[23]:


Smint = np.array(Tot_minus)
Seet  = real(np.array(Tot_excit))
Time_array=np.array(tspan)


# In[ ]:





# In[24]:



# Displaying the array


file = open(str(N_atom)+"_Results"+"_Cluster_size_"+str(Atom_cutoff)+"_Omega_"+str(Omega)+"_Lax_"+str(Sigma_x)+"_Tspan_"+str(tspan[-1])+".txt", "w+")

# Saving the array in a text file

char1="N_atom="+str(N_atom)
char2="Cluster_size="+str(Atom_cutoff)
char3="Omega/Gamma="+str(Omega)



file.write(char1)
file.write('\n')

file.write(char2)
file.write('\n')

file.write(char3)
file.write('\n')


file.write('\n')

content1 = "tspan=np.array("+str(Time_array)+")"
file.write(content1)
file.write('\n')
file.write('\n')

content2 = "Seet=np.array("+ str(Seet)+")"
file.write(content2)
file.write('\n')
file.write('\n')

content12= "Sminust==np.array("+ str(Smint)+")"
file.write(content12)
file.write('\n')
file.write('\n')

file.close()


# In[ ]:





# In[ ]:





# In[ ]:





# In[ ]:






0.16666666666666666
0.3333333333333333
0.5
0.6666666666666666
0.8333333333333334
1.0
1.1666666666666667
1.3333333333333333
1.5
1.6666666666666667
1.8333333333333333
2.0
2.1666666666666665
2.3333333333333335
2.5
2.6666666666666665
2.8333333333333335
3.0
3.1666666666666665
3.3333333333333335
3.5
3.6666666666666665
3.8333333333333335
4.0
4.166666666666667
4.333333333333333
4.5
4.666666666666667
4.833333333333333
5.0
5.166666666666667
5.333333333333333
5.5
5.666666666666667
5.833333333333333
6.0
6.166666666666667
6.333333333333333
6.5
6.666666666666667
6.833333333333333
7.0
7.166666666666667
7.333333333333333
7.5
7.666666666666667
7.833333333333333
8.0
8.166666666666666
8.333333333333334
8.5
8.666666666666666
8.833333333333334
9.0
9.166666666666666
9.333333333333334
9.5
9.666666666666666
9.833333333333334
10.0
10.166666666666666
10.333333333333334
10.5
10.666666666666666
10.833333333333334
11.0
11.166666666666666
11.333333333333334
11.5
11.666666666666666
11.833333333333334
12.0
12.1666666

In [5]:
Gij

array([[ 0.+5.00000000e-01j,  0.+7.28531033e-02j,  0.+2.40636654e-02j,
         0.+1.95871890e-02j, -0.-1.24219889e-02j, -0.-1.44608747e-02j,
        -0.-4.86443285e-03j,  0.+1.25032580e-02j,  0.+1.25535158e-02j,
         0.+8.76454264e-03j, -0.-9.46042966e-03j, -0.-8.48609768e-03j,
        -0.-1.36646570e-03j,  0.+9.55982693e-03j,  0.+8.82555451e-03j,
        -0.-8.60763341e-03j,  0.+8.33786465e-03j, -0.-3.12170293e-03j,
         0.+2.43133004e-03j,  0.+6.01971196e-03j],
       [ 0.+7.28531033e-02j,  0.+5.00000000e-01j,  0.+1.79903023e-03j,
        -0.-2.69089248e-02j,  0.+1.71373896e-02j,  0.+2.80324952e-03j,
        -0.-1.23569807e-02j, -0.-1.49723478e-02j, -0.-1.01729278e-02j,
         0.+1.19238091e-03j,  0.+8.82949028e-03j, -0.-9.45649168e-04j,
        -0.-9.10490938e-03j, -0.-7.01684970e-03j, -0.-6.49376446e-03j,
         0.+5.66323614e-03j, -0.-3.60600627e-03j,  0.+8.36438686e-03j,
        -0.-7.52289314e-03j, -0.-6.60027610e-04j],
       [ 0.+2.40636654e-02j,  0.+1.79903023e-0